<a href="https://colab.research.google.com/github/hemil19/DataScience-3/blob/master/household_load_forecasting_CNN_LSTM_Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import datetime
from keras.models import Sequential,Model
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout,Conv1D,Flatten,MaxPooling1D,Input,TimeDistributed
from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
#reading the csv file into pandas data frame
data = pd.read_csv("household_power_consumption.csv")
#setting the Date column as the index of the data frame
data['datetime'] = data['datetime'].apply(pd.to_datetime)
data.set_index('datetime',inplace=True)

In [ ]:
data

In [ ]:
data=data.resample("M").sum()

In [ ]:
data

In [ ]:
#check the quantity of null values in each columns
data.isnull().sum()

In [ ]:
data.values[:,:-1]
data.values[:,-1].reshape(-1,1)

In [ ]:
#scale the values
dataset = data.values
scaler1 = MinMaxScaler(feature_range=(0, 1))
dataset1 = scaler1.fit_transform(data.values[:,:-1])
scaler2 = MinMaxScaler(feature_range=(0, 1))
dataset2 = scaler2.fit_transform(data.values[:,-1].reshape(-1,1))

In [ ]:
dataset1.shape,dataset2.shape

In [ ]:
data.values[:,-1].reshape(-1,1)

In [ ]:
np.concatenate([dataset1, data.values[:,-1].reshape(-1,1)],axis=1)

In [ ]:
dataset=np.concatenate([dataset1, data.values[:,-1].reshape(-1,1)], axis=1)

In [ ]:
dataset

In [ ]:
#divide the data into train and test data
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
train.shape,test.shape

In [ ]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
n_steps=5
n_features=7
X_train,y_train=split_sequences(train,n_steps)
X_test,y_test=split_sequences(test,n_steps)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

In [ ]:
BATCH_SIZE=2
TRAIN_STEP = math.ceil(X_train.shape[0] / BATCH_SIZE)
VALIDATION_STEP = math.ceil(X_test.shape[0] / BATCH_SIZE)

In [ ]:
for i in range(2):
	print(X_train[i], y_train[i])

In [ ]:
X_train.shape[-2:]

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5,
                           strides=1,
                           padding="causal",
                           activation="relu",
                           input_shape=X_train.shape[-2:]),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding="valid"),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding="valid"),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(1e-4,
                                                             decay_steps=100000,
                                                             decay_rate=0.98,
                                                             staircase=False)

model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.8),
              metrics=['mae'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=500, 
                    batch_size=BATCH_SIZE,
                    steps_per_epoch=TRAIN_STEP,
                    validation_data=(X_test,y_test),
                    validation_steps=VALIDATION_STEP,
                    verbose=2, 
                    shuffle=False)

In [ ]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(10))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=250, batch_size=10, validation_data=(X_test,y_test), verbose=2, shuffle=False)

In [ ]:
# model1 = Sequential()
# model1.add(Conv1D(500, kernel_size=2, input_shape=(train_X.shape[1],train_X.shape[2])))
# model1.add(MaxPooling1D(pool_size=1))
# model1.add(LSTM(10))
# model1.add(Dense(100))
# model1.add(Dense(1,activation='sigmoid'))
# model1.compile(loss='mae', optimizer='adam')
# print(model1.summary())

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
X_train.shape

In [ ]:
#prediction on training and testing data
train_predict = model.predict(X_train)  
test_predict = model.predict(X_test)       

# #converting from three dimension to two dimension
# train_X = train_X.reshape((X_train.shape[0], X_train.shape[2]))
# test_X = test_X.reshape((X_test.shape[0], X_test.shape[2]))

# inv_train_predict = concatenate((train_predict, X_train), axis=1)
# inv_test_predict = concatenate((test_predict, X_test), axis=1)

# #transforming to original scale
# inv_train_predict = scaler.inverse_transform(inv_train_predict)
# inv_test_predict = scaler.inverse_transform(inv_test_predict)

# #predicted values on training data
# inv_train_predict = inv_train_predict[:,0]
# print("Train Predicted Values:",inv_train_predict)

# #predicted values on testing data
# inv_test_predict = inv_test_predict[:,0] 
# print("Test Predicted Values:",inv_test_predict)

In [ ]:
train_predict.shape,test_predict.shape

In [ ]:
actual=scaler2.inverse_transform(y_test.reshape(-1,1))

In [ ]:
pred=scaler2.inverse_transform(test_predict)

In [ ]:
actual.shape,pred.shape

In [ ]:
df=pd.DataFrame()
df['index']=[i for i in range(actual.shape[0])][:30]
df['actual']=actual[:30]
df['predicted']=pred[:30]
df

In [ ]:
mape=np.sum(np.abs((df['actual']-df['predicted'])/df['actual']))/len(df)
mape*100

In [ ]:
plt.figure(figsize=(25, 10))
#plt.plot(train_date, inv_train_predict,label="actual_train")
plt.plot(df['index'], df['predicted'], color='r',label="predicted")
plt.plot(df['index'], df['actual'], color='b',label="actual_test")
plt.legend(loc='best', fontsize='xx-large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.show()